In [1]:
import torch
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import random
%matplotlib inline

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import glob
import os.path
from tqdm import tqdm

In [3]:
predictionDB = pd.read_csv("../data/processed/predictionDB.csv",lineterminator='\n')

embeddings = [None]*len(predictionDB)
i=0
k=0
#for np_name in glob.glob('./../data/processed/embeddings/*.np[yz]'):
#    embeddings[i] = np.load(np_name)
#    i = i + 1

for x in predictionDB["COMMIT_HASH"]:
    embeddings[i] = np.load("./../data/processed/embeddings/"+x+".npy")
    i = i + 1

embeddings
print(len(predictionDB),i)

62917 62917


In [29]:
clean = ''
for i,s in enumerate(predictionDB["COMMIT_MESSAGE"][3].split()[:-3]):
    if i!=0:
        clean+= ' '
    clean+=s
print(clean)
    

add some more tests PR: MRM-9


In [35]:
clean_column = [None]*len(predictionDB["COMMIT_MESSAGE"])
for i in range(len(predictionDB["COMMIT_MESSAGE"])):
    clean = ''
    for j,s in enumerate(predictionDB["COMMIT_MESSAGE"][i].split()[:-3]):
        if j!=0:
            clean+= ' '
        clean+=s
    clean_column[i] = clean
predictionDB["CLEAN_CMS"] = clean_column

In [37]:
predictionDB.to_csv("../data/processed/predictionDB2.csv", index='False') #export!

In [14]:
from scipy import spatial
a = predictionDB["COMMIT_MESSAGE"][10]
b = predictionDB["COMMIT_MESSAGE"][11]
emb_a = embeddings[10]
emb_b = embeddings[11]
print("a =",a)
print("b =",b)

a = test invalid version PR: MRM-9  git-svn-id: https://svn.apache.org/repos/asf/maven/repository-manager/trunk@349661 13f79535-47bb-0310-9956-ffa450edef68
b = test invalid snapshot version PR: MRM-9  git-svn-id: https://svn.apache.org/repos/asf/maven/repository-manager/trunk@349671 13f79535-47bb-0310-9956-ffa450edef68


In [15]:
print(spatial.distance.cosine(emb_a, emb_b))

0.08272308111190796


In [21]:
predictionDB["COMMIT_HASH"][11]

'824ecd2c8420a429cc1b9ae3dbcf58e6497b8cb6'

In [20]:
c = predictionDB["COMMIT_MESSAGE"][3333]
c

'move cargo port to a property and selenium plugin version to pluginManagement section  git-svn-id: https://svn.apache.org/repos/asf/archiva/trunk@1136207 13f79535-47bb-0310-9956-ffa450edef68'

In [5]:
embeddings2 = pd.Series( (v for v in embeddings) )
embeddings2

0        [-0.04031309, -0.06636557, -0.051734935, 0.024...
1        [-0.03422957, -0.06418772, -0.04951831, 0.0361...
2        [-0.02127257, -0.0677829, -0.050998665, 0.0245...
3        [-0.030376969, -0.06409717, -0.03432051, 0.034...
4        [-0.06506152, -0.045885205, -0.053898133, 0.03...
                               ...                        
62912    [0.012309925, -0.03879959, -0.0022678305, 0.06...
62913    [-0.02502151, -0.017792063, -0.027390199, 0.00...
62914    [-0.034420438, 0.014770305, -0.03304358, 0.047...
62915    [-0.061842874, 0.05742638, 0.038782373, -0.034...
62916    [-0.025216602, 0.0053232913, 0.03906781, 0.020...
Length: 62917, dtype: object

In [6]:
#data = embeddings
labels = predictionDB["inc_complexity"]

for i in range(len(labels)):
    if labels[i]<=0:
        labels[i]=0
    else:
        labels[i]=1

labels

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


0        0.0
1        0.0
2        1.0
3        1.0
4        1.0
        ... 
62912    1.0
62913    0.0
62914    0.0
62915    0.0
62916    1.0
Name: inc_complexity, Length: 62917, dtype: float64

In [7]:
from sklearn.model_selection import train_test_split


data_train, data_test, labels_train, labels_test = train_test_split(embeddings2, labels, test_size=0.20, random_state=42)


In [8]:
labels_train.shape

(50333,)

In [9]:
type(embeddings2)

pandas.core.series.Series

In [10]:
class commits_dataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X.index)
    
    def __getitem__(self, index):
        return torch.Tensor(self.X.iloc[index]),torch.as_tensor(self.y.iloc[index]).float()

In [11]:
commits_dataset_train = commits_dataset(X=data_train,y=labels_train)
commits_dataset_test = commits_dataset(X=data_test,y=labels_test)
# print(commits_dataset_train[0])

In [12]:
train_loader = DataLoader(dataset=commits_dataset_train, batch_size=32, shuffle=True)
valid_loader = DataLoader(dataset=commits_dataset_test, batch_size=32, shuffle=False)
#dls = DataLoaders(train_loader,valid_loader)


In [13]:
#predictionDB["is_valid"] = np.zeros(len(predictionDB))
#for i in range(len(predictionDB["is_valid"])):
#    predictionDB["is_valid"][i] = 1 if random.random()<0.2 else 0 

In [14]:
#from fastai.text.all import *
#dls = TextDataLoaders.from_df(predictionDB, text_col='COMMIT_MESSAGE', label_col='inc_complexity', valid_col='is_valid')
#dls.show_batch(max_n=3)

In [31]:
# Multilayer perceptron
class MultilayerPerceptron(nn.Module):
    def __init__(self):
        super().__init__()
        self.lin1 = nn.Linear(384, 512, bias=True) 
        self.lin2 = nn.Linear(512, 256, bias=True)
        self.lin3 = nn.Linear(256, 1, bias=True)

    def forward(self, xb):
        x = xb.float()
        #x = xb.view(250, -1)
        x = F.relu(self.lin1(x))
        x = F.relu(self.lin2(x))
        return self.lin3(x)

In [32]:
#mlp_learner = Learner(data=data, model=MultilayerPerceptron(), loss_func=nn.CrossEntropyLoss(),metrics=accuracy)
#mlp_learner.fine_tune(20)

In [33]:
model = MultilayerPerceptron()
print(model)

MultilayerPerceptron(
  (lin1): Linear(in_features=384, out_features=512, bias=True)
  (lin2): Linear(in_features=512, out_features=256, bias=True)
  (lin3): Linear(in_features=256, out_features=1, bias=True)
)


In [34]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
loss_fn = nn.BCELoss()

In [35]:
mean_train_losses = []
mean_valid_losses = []
valid_acc_list = []
epochs = 100

for epoch in range(epochs):
    model.train()
    
    train_losses = []
    valid_losses = []
    for i, (embeddings, labels) in tqdm(enumerate(train_loader)):
        
        optimizer.zero_grad()
        
        outputs = model(embeddings)
        loss = loss_fn(outputs.squeeze(0),labels)
        loss.backward()
        optimizer.step()
        
        train_losses.append(loss.item())
        
            
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for i, (embeddings, labels) in enumerate(valid_loader):
            outputs = model(embeddings)
            loss = loss_fn(outputs.squeeze(0), labels)
            
            valid_losses.append(loss.item())
            
            _, predicted = torch.max(outputs.data, 1)
            
            total += labels.size(0)
            
    mean_train_losses.append(np.mean(train_losses))
    mean_valid_losses.append(np.mean(valid_losses))
    print('epoch : {}, train loss : {:.4f}, valid loss : {:.4f}'\
         .format(epoch+1, mean_train_losses[-1], mean_valid_losses[-1]))

0it [00:00, ?it/s]


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x12288 and 384x512)